《设计模式：可复用面向对象软件的基础》
- 对接口编程，而不是对实现编程
- 优先使用对象组合，而不是类继承

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

# 上下文
# 把一些计算委托给实现不同算法的可互换组件，提供服务
class Order:

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# 策略：抽象基类
# 实现不同算法的组件共同的接口
class Promotion(ABC):

    @abstractmethod
    def discount(self, order):
        """返回折扣金额（正数）"""


# 具体策略
class FidelityPromo(Promotion):
    """为积分 100 或以上的顾客提供 5% 的折扣"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


# 具体策略
class BulkItemPromo(Promotion):
    """单个商品 20 个或以上时提供 10% 折扣"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

# 具体策略
class LargeOrderPromo(Promotion):
    """订单中的不同商品达到 10 个或以上时提供 7% 折扣"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [2]:
# 顾客，积分
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

# 购物车商品
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [3]:
# 无折扣
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [4]:
# 5% 折扣
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [5]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

# 香蕉 10% 优惠
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [6]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

# 整个订单 7% 折扣
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [7]:
# 把函数作为对象

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # 上下文

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)  # 计算折扣只需调用 self.promotion() 函数
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# 没有抽象基类
# 各个策略都是函数

def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [8]:
# 顾客，积分
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

# 购物车商品
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [9]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [10]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [11]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

# 香蕉 10% 优惠
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [12]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

# 整个订单 7% 折扣
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [13]:
# 迭代函数列表，找出最大的折扣额度
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

# Order 实例
def best_promo(order):
    """选择可用的最佳折扣
    """
    
    # 返回最大折扣额度
    return max(promo(order) for promo in promos)

In [14]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [15]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [16]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [17]:
# 在 Python 中，模块也是一等对象，而且标准库提供了几个处理模块的函数
# globals() 返回当前的全局符号表的字典
promos = [globals()[name] for name in globals()  # 返回字典中的各个 name
            if name.endswith('_promo')  # 只选择以 _promo 结尾的
            and name != 'best_promo']   # 过滤 best_promo 本身

# Order 实例
def best_promo(order):
    """选择可用的最佳折扣
    """
    
    # 返回最大折扣额度
    return max(promo(order) for promo in promos)

In [18]:
# 内省名为 promotions 的独立模块
# inspect.getmembers 函数用于获取对象的属性，第二个参数是可选的判断条件
import inspect
import promotions
promos = [func for name, func in
                inspect.getmembers(promotions, inspect.isfunction)]

# Order 实例
def best_promo(order):
    """选择可用的最佳折扣
    """
    
    # 返回最大折扣额度
    return max(promo(order) for promo in promos)

In [19]:
# 可调用实例，可以保存任何所需的状态
# 可以使用闭包在调用之间保存函数的内部状态
class MacroCommand:
    """一个执行一组命令的命令"""
    
    def __init__(self, commands):
        self.commands = list(commands)
    
    def __call__(self):
        for command in self.commands:
            command()